In [57]:
import time
start = time.time()
import os
import ROOT
ROOT.gSystem.Load("libGenVector")
from ROOT import RDataFrame, RVec
from ROOT import TChain, TSelector, TTree, TH1F, TCanvas, TFile, TEfficiency, TLegend,TLine
from ROOT import Math
from ROOT import TLorentzVector
from ROOT import TVector3
from array import array
import numpy as np

ROOT.gInterpreter.Reset()

ROOT.EnableImplicitMT()
print(ROOT.ROOT.IsImplicitMTEnabled())
ROOT.gROOT.SetBatch(ROOT.kTRUE)
folder="/eos/user/f/fsimone/B_Dh_summerproject/bphnano/B_to_Dpi_KKpi_Run3Summer24NanoAOD_v0_2025Aug27/B_to_Dpi_KKpi__TuneCP5_13p6TeV_pythia8_Run3Summer24GS_v1/B_to_Dpi_KKpi__Run3Summer24NanoAOD_v0/250827_105118/"
tree=TChain("Events")
for fol,subfolders,files in os.walk(folder):
    for file in files:
        if file.endswith(".root"):
            tree.Add(fol+"/"+file)


df = RDataFrame(tree)
decay="KKpi"
id1 = [321]
id2 = [-321]
id3 = 211
t = False
th1 = 0.01
th2 = 0.01
th3 = 0.01

cpp_code = """
#include <ROOT/RVec.hxx>
#include <Math/Vector4Dfwd.h>
#include <Math/VectorUtil.h>
#include <TDatabasePDG.h>
#include <TMath.h>
#include <map>
#include <set>
#include <tuple>
#include <vector>
#include <algorithm>
#include <iterator>
#include <utility>

using namespace ROOT;
using namespace ROOT::Math;

struct GenParticle {
    PtEtaPhiMVector p4;
    int midx;
    int typ;
    int pdgId;
    float pt;
    float dist;
};

using Matched = std::tuple<PtEtaPhiMVector, int, PtEtaPhiMVector>;

struct Results {
    std::vector<RVec<float>> vecs;
    float muon_count;
    float reco_count;
    float acc_count;
};

std::vector<Matched> matching_m(const RVec<float>& g_eta, const RVec<float>& g_phi, const RVec<float>& g_pt, const RVec<float>& g_mass, const RVec<int>& g_midx,
                                  const std::vector<Matched>& p, const std::vector<Matched>& m,
                                  RVec<float>& hDen, RVec<float>& hNum, RVec<float>& hdR, float th, RVec<float>& hIM) {
    std::vector<Matched> matches;
    for (const auto& plus : p) {
        for (const auto& minus_ : m) {
            if (std::get<1>(plus) == std::get<1>(minus_)) {
                int idx = std::get<1>(plus);
                PtEtaPhiMVector GenMother(g_pt[idx], g_eta[idx], g_phi[idx], g_mass[idx]);
                PtEtaPhiMVector RecoMother = std::get<2>(plus) + std::get<2>(minus_);
                float dr = VectorUtil::DeltaR(GenMother, RecoMother);
                hIM.push_back(RecoMother.M());
                hdR.push_back(dr);
                hNum.push_back(g_pt[idx]);
                matches.emplace_back(GenMother, g_midx[idx], RecoMother);
            }
        }
    }
    return matches;
}


void matching_func(std::vector<GenParticle>& Gp, std::vector<PtEtaPhiMVector>& Tp,
                   RVec<float>& d1_num_pt, RVec<float>& d2_num_pt, RVec<float>& d3_num_pt,
                   RVec<float>& d1_num_d, RVec<float>& d2_num_d, RVec<float>& d3_num_d,
                   RVec<float>& d1_dr, RVec<float>& d2_dr, RVec<float>& d3_dr,
                   std::vector<Matched>& m1, std::vector<Matched>& m2, std::vector<Matched>& m3,
                   float th1, float th2, float th3) {
    TDatabasePDG *pdg_db = TDatabasePDG::Instance();
    bool go = true;
    while (go) {
        go = false;
        std::map<int, std::set<std::pair<int, float>>> match_map;
        int i = 0;
        while (i < Gp.size()) {
            float dR0 = 1000.0f;
            int idx = 0;
            for (int j = 0; j < Tp.size(); ++j) {
                float dR = VectorUtil::DeltaR(Gp[i].p4, Tp[j]);
                if (dR < dR0) {
                    dR0 = dR;
                    idx = j;
                }
            }
            int typ = Gp[i].typ;
            float th = (typ == 1 ? th1 : (typ == 2 ? th2 : th3));
            if (dR0 < th) {
                auto& s = match_map[idx];
                s.emplace(i, dR0);
                if (s.size() > 1) go = true;
                ++i;
            } else {
                if (typ == 1) d1_dr.push_back(dR0);
                else if (typ == 2) d2_dr.push_back(dR0);
                else d3_dr.push_back(dR0);
                Gp.erase(Gp.begin() + i);
            }
        }
        std::vector<int> l_g, l_t;
        for (const auto& kv : match_map) {
            int e = kv.first;
            const auto& s = kv.second;
            float dr = 1000.0f;
            int ind = 0;
            for (const auto& el : s) {
                if (el.second < dr) {
                    dr = el.second;
                    ind = el.first;
                }
            }
            int typ = Gp[ind].typ;
            int pdgid = Gp[ind].pdgId;
            double mass = pdg_db->GetParticle(pdgid)->Mass();
            Tp[e] = PtEtaPhiMVector(Tp[e].Pt(), Tp[e].Eta(), Tp[e].Phi(), mass);
            if (typ == 1) {
                d1_num_pt.push_back(Gp[ind].pt);
                d1_dr.push_back(dr);
                d1_num_d.push_back(Gp[ind].dist);
                m1.emplace_back(Gp[ind].p4, Gp[ind].midx, Tp[e]);
            } else if (typ == 2) {
                d2_num_pt.push_back(Gp[ind].pt);
                d2_dr.push_back(dr);
                d2_num_d.push_back(Gp[ind].dist);
                m2.emplace_back(Gp[ind].p4, Gp[ind].midx, Tp[e]);
            } else {
                d3_num_pt.push_back(Gp[ind].pt);
                d3_dr.push_back(dr);
                d3_num_d.push_back(Gp[ind].dist);
                m3.emplace_back(Gp[ind].p4, Gp[ind].midx, Tp[e]);
            }
            l_g.push_back(ind);
            l_t.push_back(e);
        }
        std::sort(l_g.begin(), l_g.end(), std::greater<int>());
        std::sort(l_t.begin(), l_t.end(), std::greater<int>());
        for (auto eg : l_g) {
            Gp.erase(Gp.begin() + eg);
        }
        for (auto et : l_t) {
            Tp.erase(Tp.begin() + et);
        }
    }
}

Results main_func(const RVec<float>& g_pt, const RVec<float>& g_eta, const RVec<float>& g_phi, const RVec<float>& g_mass,
                  const RVec<int>& g_id, const RVec<int>& st, const RVec<int>& g_midx,
                  const RVec<float>& t_pt, const RVec<float>& t_eta, const RVec<float>& t_phi, const RVec<int>& t_ch, const RVec<float>& t_mass,
                  const RVec<float>& g_vx, const RVec<float>& g_vy, const RVec<float>& g_vz,
                  const RVec<float>& m_eta, const RVec<float>& m_pt, bool t_opt,
                  const RVec<int>& id1, const RVec<int>& id2, int id3,
                  float th1, float th2, float th3) {
    Results res;
    res.muon_count = 0;
    res.reco_count = 0;
    res.acc_count = 0;
    TDatabasePDG *pdg = TDatabasePDG::Instance();
    std::map<int, std::vector<GenParticle>> d;
    std::vector<GenParticle> Gp, Gm;
    RVec<float> d1_num_pt, d2_num_pt, d3_num_pt, d1_num_d, d2_num_d, d3_num_d, d1_den_pt, d2_den_pt, d3_den_pt, d1_den_d, d2_den_d, d3_den_d, d1_dr, d2_dr, d3_dr;
    RVec<float> B_num_pt, D_num_pt, B_den_pt, D_den_pt, B_dr, D_dr, B_M, D_M;
    RVec<float> d1_eta, d2_eta, d3_eta, d1_phi, d2_phi, d3_phi, d1_gen_eta, d2_gen_eta, d3_gen_eta, d1_gen_phi, d2_gen_phi, d3_gen_phi, d1_pt, d2_pt, d3_pt;
    bool go = false;
    if (t_opt) {
        for (size_t im = 0; im < m_eta.size(); ++im) {
            if (std::abs(m_eta[im]) < 1.5 && m_pt[im] > 9) {
                go = true;
                break;
            }
        }
    } else {
        go = true;
    }

    auto is_in = [](int id, const RVec<int>& vec) {
        return std::find(vec.begin(), vec.end(), id) != vec.end();
    };

    for (size_t i = 0; i < g_id.size(); ++i) {
        if (abs(g_id[i]) == 13 && g_pt[i] > 9 && std::abs(g_eta[i]) < 1.5){
            res.muon_count = 1;
        }
        if (st[i] == 1 && g_pt[i] > 0.5 && std::abs(g_eta[i]) < 2.4) {
            int midx = g_midx[i];
            if (is_in(g_id[i], id1) && std::abs(g_id[midx]) == 421) {
                int gmidx = g_midx[midx];
                if (std::abs(g_id[gmidx]) == 521) {
                    PtEtaPhiMVector GenP(g_pt[i], g_eta[i], g_phi[i], pdg->GetParticle(g_id[i])->Mass());
                    float dist = TMath::Sqrt(g_vx[i] * g_vx[i] + g_vy[i] * g_vy[i] + g_vz[i] * g_vz[i]);
                    d[gmidx].push_back({GenP, midx, 1, g_id[i], g_pt[i], dist});
                }
            } else if (is_in(g_id[i], id2) && std::abs(g_id[midx]) == 421) {
                int gmidx = g_midx[midx];
                if (std::abs(g_id[gmidx]) == 521) {
                    PtEtaPhiMVector GenP(g_pt[i], g_eta[i], g_phi[i], pdg->GetParticle(g_id[i])->Mass());
                    float dist = TMath::Sqrt(g_vx[i] * g_vx[i] + g_vy[i] * g_vy[i] + g_vz[i] * g_vz[i]);
                    d[gmidx].push_back({GenP, midx, 2, g_id[i], g_pt[i], dist});
                }
            } else if (g_id[i] == -id3 && g_id[midx] == -521) {
                PtEtaPhiMVector GenP(g_pt[i], g_eta[i], g_phi[i], pdg->GetParticle(g_id[i])->Mass());
                float dist = TMath::Sqrt(g_vx[i] * g_vx[i] + g_vy[i] * g_vy[i] + g_vz[i] * g_vz[i]);
                d[midx].push_back({GenP, midx, 3, g_id[i], g_pt[i], dist});
            } else if (g_id[i] == id3 && g_id[midx] == 521) {
                PtEtaPhiMVector GenP(g_pt[i], g_eta[i], g_phi[i], pdg->GetParticle(g_id[i])->Mass());
                float dist = TMath::Sqrt(g_vx[i] * g_vx[i] + g_vy[i] * g_vy[i] + g_vz[i] * g_vz[i]);
                d[midx].push_back({GenP, midx, 3, g_id[i], g_pt[i], dist});
            }
        }
    }
    for (const auto& pair : d) {
        int B = pair.first;
        const auto& parts = pair.second;
        int c1 = 0; int c2 = 0; int c3 = 0;
        for (const auto& el : parts){
            if (el.typ == 1) c1++;
            if (el.typ == 2) c2++;
            if (el.typ == 3) c3++;
        }
        if (c1 == 1 && c2 == 1 && c3 == 1) {
            GenParticle d_1, d_2, d_3;
            res.acc_count = 1;
            for (const auto& e : parts) {
                int typ = e.typ;
                if (typ == 1) {
                    d_1 = e;
                    B_den_pt.push_back(g_pt[B]);
                    D_den_pt.push_back(g_pt[e.midx]);
                } else if (typ == 2) {
                    d_2 = e;
                } else if (typ == 3){
                    d_3 = e;
                    d3_den_pt.push_back(d_3.pt);
                    d3_den_d.push_back(d_3.dist);
                    d3_gen_eta.push_back(d_3.p4.Eta());
                    d3_gen_phi.push_back(d_3.p4.Phi());
                    if (pdg->GetParticle(d_3.pdgId)->Charge() > 0) Gp.push_back(d_3);
                    else Gm.push_back(d_3);
                }
            }
            if (d_1.pt < d_2.pt){
                d1_den_pt.push_back(d_1.pt); d1_den_d.push_back(d_1.dist); d1_gen_eta.push_back(d_1.p4.Eta()); d1_gen_phi.push_back(d_1.p4.Phi());
                if (pdg->GetParticle(d_1.pdgId)->Charge() > 0) Gp.push_back(d_1); else Gm.push_back(d_1);

                d2_den_pt.push_back(d_2.pt); d2_den_d.push_back(d_2.dist); d2_gen_eta.push_back(d_2.p4.Eta()); d2_gen_phi.push_back(d_2.p4.Phi());
                if (pdg->GetParticle(d_2.pdgId)->Charge() > 0) Gp.push_back(d_2); else Gm.push_back(d_2);
            } else {
                d_1.typ = 2; d_2.typ = 1;
                d1_den_pt.push_back(d_2.pt); d1_den_d.push_back(d_2.dist); d1_gen_eta.push_back(d_2.p4.Eta()); d1_gen_phi.push_back(d_2.p4.Phi());
                if (pdg->GetParticle(d_2.pdgId)->Charge() > 0) Gp.push_back(d_2); else Gm.push_back(d_2);

                d2_den_pt.push_back(d_1.pt); d2_den_d.push_back(d_1.dist); d2_gen_eta.push_back(d_1.p4.Eta()); d2_gen_phi.push_back(d_1.p4.Phi());
                if (pdg->GetParticle(d_1.pdgId)->Charge() > 0) Gp.push_back(d_1); else Gm.push_back(d_1);
            }
        }
    }
    if (go) {
        std::vector<PtEtaPhiMVector> Tp, Tm;
        for (size_t it = 0; it < t_ch.size(); ++it) {
            if ( t_pt[it] > 0.5 && std::abs(t_eta[it])<2.4 ) {
                PtEtaPhiMVector Track(t_pt[it], t_eta[it], t_phi[it], t_mass[it]);
                if (t_ch[it] == 1 ) Tp.push_back(Track);
                else if (t_ch[it] == -1) Tm.push_back(Track);
            }
        }
        std::vector<Matched> m1a, m2a, m3a;
        matching_func(Gp, Tp, d1_num_pt, d2_num_pt, d3_num_pt, d1_num_d, d2_num_d, d3_num_d, d1_dr, d2_dr, d3_dr, m1a, m2a, m3a, th1, th2, th3);
        std::vector<Matched> m1b, m2b, m3b;
        matching_func(Gm, Tm, d1_num_pt, d2_num_pt, d3_num_pt, d1_num_d, d2_num_d, d3_num_d, d1_dr, d2_dr, d3_dr, m1b, m2b, m3b, th1, th2, th3);

        std::vector<Matched> m1; m1.reserve(m1a.size() + m1b.size()); m1.insert(m1.end(), m1a.begin(), m1a.end()); m1.insert(m1.end(), m1b.begin(), m1b.end());
        std::vector<Matched> m2; m2.reserve(m2a.size() + m2b.size()); m2.insert(m2.end(), m2a.begin(), m2a.end()); m2.insert(m2.end(), m2b.begin(), m2b.end());
        std::vector<Matched> m3; m3.reserve(m3a.size() + m3b.size()); m3.insert(m3.end(), m3a.begin(), m3a.end()); m3.insert(m3.end(), m3b.begin(), m3b.end());
        for (const auto& e : m1) { const auto& reco = std::get<2>(e); d1_pt.push_back(reco.Pt()); d1_eta.push_back(reco.Eta()); d1_phi.push_back(reco.Phi()); }
        for (const auto& e : m2) { const auto& reco = std::get<2>(e); d2_pt.push_back(reco.Pt()); d2_eta.push_back(reco.Eta()); d2_phi.push_back(reco.Phi()); }
        for (const auto& e : m3) { const auto& reco = std::get<2>(e); d3_pt.push_back(reco.Pt()); d3_eta.push_back(reco.Eta()); d3_phi.push_back(reco.Phi()); }
        std::vector<Matched> mD = matching_m(g_eta, g_phi, g_pt, g_mass, g_midx, m1, m2, D_den_pt, D_num_pt, D_dr, 0.05f, D_M);
        std::vector<Matched> mB = matching_m(g_eta, g_phi, g_pt, g_mass, g_midx, mD, m3, B_den_pt, B_num_pt, B_dr, 0.05f, B_M);
        if (mB.size() > 0){
            res.reco_count = 1;
        }
    }
    res.vecs.reserve(38);
    res.vecs = {d1_num_pt, d2_num_pt, d3_num_pt, d1_num_d, d2_num_d, d3_num_d, d1_den_pt, d2_den_pt, d3_den_pt, d1_den_d, d2_den_d, d3_den_d, d1_dr, d2_dr, d3_dr, B_num_pt, D_num_pt, B_den_pt, D_den_pt, B_dr, D_dr, B_M, D_M, d1_eta, d2_eta, d3_eta, d1_phi, d2_phi, d3_phi, d1_pt, d2_pt, d3_pt, d1_gen_phi, d2_gen_phi, d3_gen_phi, d1_gen_eta, d2_gen_eta, d3_gen_eta};
    return res;
}
"""

ROOT.gInterpreter.Declare(cpp_code)

id1_cpp = f"ROOT::RVec<int>{{{', '.join(map(str, id1))}}}"
id2_cpp = f"ROOT::RVec<int>{{{', '.join(map(str, id2))}}}"

df1 = df.Define("results", f"main_func(GenPart_pt, GenPart_eta, GenPart_phi, GenPart_mass, GenPart_pdgId, GenPart_status, GenPart_genPartIdxMother, Track_pt, Track_eta, Track_phi, Track_charge, Track_mass, GenPart_vx, GenPart_vy, GenPart_vz, AllMuon_eta, AllMuon_pt, {1 if t else 0}, {id1_cpp}, {id2_cpp}, {id3}, {th1}, {th2}, {th3})") \
        .Define("d1_num_pt", "results.vecs[0]") \
        .Define("d2_num_pt", "results.vecs[1]") \
        .Define("d3_num_pt", "results.vecs[2]") \
        .Define("d1_num_d", "results.vecs[3]") \
        .Define("d2_num_d", "results.vecs[4]") \
        .Define("d3_num_d", "results.vecs[5]") \
        .Define("d1_den_pt", "results.vecs[6]") \
        .Define("d2_den_pt", "results.vecs[7]") \
        .Define("d3_den_pt", "results.vecs[8]") \
        .Define("d1_den_d", "results.vecs[9]") \
        .Define("d2_den_d", "results.vecs[10]") \
        .Define("d3_den_d", "results.vecs[11]") \
        .Define("d1_dr", "results.vecs[12]") \
        .Define("d2_dr", "results.vecs[13]") \
        .Define("d3_dr", "results.vecs[14]") \
        .Define("B_num_pt", "results.vecs[15]") \
        .Define("D_num_pt", "results.vecs[16]") \
        .Define("B_den_pt", "results.vecs[17]") \
        .Define("D_den_pt", "results.vecs[18]") \
        .Define("B_dr", "results.vecs[19]") \
        .Define("D_dr", "results.vecs[20]") \
        .Define("B_M", "results.vecs[21]") \
        .Define("D_M", "results.vecs[22]") \
        .Define("d1_eta", "results.vecs[23]") \
        .Define("d2_eta", "results.vecs[24]") \
        .Define("d3_eta", "results.vecs[25]") \
        .Define("d1_phi", "results.vecs[26]") \
        .Define("d2_phi", "results.vecs[27]") \
        .Define("d3_phi", "results.vecs[28]") \
        .Define("d1_pt", "results.vecs[29]") \
        .Define("d2_pt", "results.vecs[30]") \
        .Define("d3_pt", "results.vecs[31]") \
        .Define("d1_gen_phi", "results.vecs[32]") \
        .Define("d2_gen_phi", "results.vecs[33]") \
        .Define("d3_gen_phi", "results.vecs[34]") \
        .Define("d1_gen_eta", "results.vecs[35]") \
        .Define("d2_gen_eta", "results.vecs[36]") \
        .Define("d3_gen_eta", "results.vecs[37]") \
        .Define("muon_counter", "results.muon_count")\
        .Define("reco_counter", "results.reco_count")\
        .Define("acc_counter", "results.acc_count")

ROOT.gErrorIgnoreLevel = ROOT.kError

branches = ["d1_num_pt", "d2_num_pt", "d3_num_pt", "d1_den_pt", "d2_den_pt", "d3_den_pt", "d1_dr", "d2_dr", "d3_dr", "B_num_pt", "D_num_pt", "B_den_pt", "D_den_pt", "B_dr", "D_dr","B_M","D_M","d1_pt","d2_pt","d3_pt","d1_eta","d2_eta","d3_eta","d1_phi","d2_phi","d3_phi","d1_gen_eta","d2_gen_eta","d3_gen_eta","d1_gen_phi","d2_gen_phi","d3_gen_phi"]

binning = {
    "d1_eta": (100, -2.5, 2.5), "d1_phi": (100, -4, 4), "d2_eta": (100, -2.5, 2.5),
    "d2_phi": (100, -4, 4), "d3_eta": (100, -2.5, 2.5), "d3_phi": (100, -4, 4),
    "d1_dr": (100,0,0.05), "d2_dr": (100,0,0.05), "d3_dr": (100,0,0.05),
    "B_dr": (100,0,0.1), "D_dr": (100,0,0.1), "B_M": (100,4.8,5.8), "D_M": (100,1.5,2.25),
    "d1_gen_eta": (100, -2.5, 2.5), "d1_gen_phi": (100, -4, 4), "d2_gen_eta": (100, -2.5, 2.5),
    "d2_gen_phi": (100, -4, 4), "d3_gen_eta": (100, -2.5, 2.5), "d3_gen_phi": (100, -4, 4)
}

histo_pointers = {}
print("Booking all histograms...")
for branch in branches:
    nbins, min_val, max_val = binning.get(branch, (100, 0, 25))
    model = ROOT.RDF.TH1DModel(f"hist_{branch}", f"{branch};Value;Entries", nbins, min_val, max_val)
    histo_pointers[branch] = df1.Histo1D(model, branch)

model = ROOT.RDF.TH2DModel("hist_eta_phi_reco_final_state_1", "eta_phi_reco_distribution_final_state_1;Eta;Phi", 100, -2.5, 2.5,100,-4,4)
histo_pointers["eta_phi_reco_final_state_1"] = df1.Histo2D(model,"d1_eta","d1_phi")
model = ROOT.RDF.TH2DModel("hist_eta_phi_reco_final_state_2", "eta_phi_reco_distribution_final_state_2;Eta;Phi", 100, -2.5, 2.5,100,-4,4)
histo_pointers["eta_phi_reco_final_state_2"] = df1.Histo2D(model,"d2_eta","d2_phi")
model = ROOT.RDF.TH2DModel("hist_eta_phi_reco_final_state_3", "eta_phi_reco_distribution_final_state_3;Eta;Phi", 100, -2.5, 2.5,100,-4,4)
histo_pointers["eta_phi_reco_final_state_3"] = df1.Histo2D(model,"d3_eta","d3_phi")
model = ROOT.RDF.TH2DModel("hist_eta_phi_gen_final_state_1", "eta_gen_distribution_final_state_1;Eta;Phi", 100, -2.5, 2.5,100,-4,4)
histo_pointers["eta_phi_gen_final_state_1"] = df1.Histo2D(model,"d1_gen_eta","d1_gen_phi")
model = ROOT.RDF.TH2DModel("hist_eta_phi_gen_final_state_2", "eta_gen_distribution_final_state_2;Eta;Phi", 100, -2.5, 2.5,100,-4,4)
histo_pointers["eta_phi_gen_final_state_2"] = df1.Histo2D(model,"d2_gen_eta","d2_gen_phi")
model = ROOT.RDF.TH2DModel("hist_eta_phi_gen_final_state_3", "eta_gen_distribution_final_state_3;Eta;Phi", 100, -2.5, 2.5,100,-4,4)
histo_pointers["eta_phi_gen_final_state_3"] = df1.Histo2D(model,"d3_gen_eta","d3_gen_phi")

muon_count_ptr = df1.Sum("muon_counter")
reco_count_ptr = df1.Sum("reco_counter")
acc_count_ptr = df1.Sum("acc_counter")
total_ptr = df1.Count()

print("Triggering event loop to fill all histograms and compute sums...")
dummy = list(histo_pointers.values())[0].GetValue()

f = {}
for branch, ptr in histo_pointers.items():
    print(f"Retrieving histogram for {branch}...")
    f[branch] = ptr.GetValue()

muon_count = muon_count_ptr.GetValue()
reco_count = reco_count_ptr.GetValue()
acc_count = acc_count_ptr.GetValue()
total = total_ptr.GetValue()
print("total evets:",total,"-trigger events number:",muon_count,"-reconstructed events number:",reco_count,"-acceptance events number:",acc_count)

print("Saving histograms to file...")
out_file = ROOT.TFile("histograms"+decay+".root", "RECREATE")
for hist in f.values():
    hist.Write()
out_file.Close()
print(f"All histograms saved in histograms{decay}.root")
end = time.time()
print(f"Tempo di esecuzione: {end - start:.3f} secondi")

True
Booking all histograms...
Triggering event loop to fill all histograms and compute sums...
Retrieving histogram for d1_num_pt...
Retrieving histogram for d2_num_pt...
Retrieving histogram for d3_num_pt...
Retrieving histogram for d1_den_pt...
Retrieving histogram for d2_den_pt...
Retrieving histogram for d3_den_pt...
Retrieving histogram for d1_dr...
Retrieving histogram for d2_dr...
Retrieving histogram for d3_dr...
Retrieving histogram for B_num_pt...
Retrieving histogram for D_num_pt...
Retrieving histogram for B_den_pt...
Retrieving histogram for D_den_pt...
Retrieving histogram for B_dr...
Retrieving histogram for D_dr...
Retrieving histogram for B_M...
Retrieving histogram for D_M...
Retrieving histogram for d1_pt...
Retrieving histogram for d2_pt...
Retrieving histogram for d3_pt...
Retrieving histogram for d1_eta...
Retrieving histogram for d2_eta...
Retrieving histogram for d3_eta...
Retrieving histogram for d1_phi...
Retrieving histogram for d2_phi...
Retrieving histogra

input_line_240:18:8: error: redefinition of 'GenParticle'
struct GenParticle {
       ^
input_line_71:18:8: note: previous definition is here
struct GenParticle {
       ^
input_line_240:29:8: error: redefinition of 'Results'
struct Results {
       ^
input_line_71:29:8: note: previous definition is here
struct Results {
       ^
input_line_240:36:22: error: redefinition of 'matching_m'
std::vector<Matched> matching_m(const RVec<float>& g_eta, const RVec<float>& g_phi, const RVec<float>& g_pt, const RVec<float>& g_mass, const RVec<int>& g_midx,
                     ^
input_line_71:36:22: note: previous definition is here
std::vector<Matched> matching_m(const RVec<float>& g_eta, const RVec<float>& g_phi, const RVec<float>& g_pt, const RVec<float>& g_mass, const RVec<int>& g_midx,
                     ^
input_line_240:58:6: error: redefinition of 'matching_func'
void matching_func(std::vector<GenParticle>& Gp, std::vector<PtEtaPhiMVector>& Tp,
     ^
input_line_71:58:6: note: previous de

In [58]:
name="Plots_"+decay
os.system("mkdir -p "+name)

0

In [59]:
c_eff = TCanvas("c_eff", "lower pt D daughter Efficiency", 700, 500)
num = f["d1_num_pt"]
den = f["d1_den_pt"]
eff = ROOT.TEfficiency(num, den)
eff.SetTitle("("+decay+")"+"Efficiency of lower pt D-daughter vs pt ; p_{t} [GeV/c]; Efficiency")
eff.Draw()
c_eff.SaveAs(name+"/efficiency_d1_vs_pt.png")
c_eff.Draw()

In [60]:
c_eff = TCanvas("c_eff", "higher pt D daughter Efficiency", 700, 500)
num = f["d2_num_pt"]
den = f["d2_den_pt"]
eff = ROOT.TEfficiency(num, den)
eff.SetTitle("("+decay+")"+"Efficiency of higher pt D-daughter vs pt; p_{t} [GeV]; Efficiency")
eff.Draw()
c_eff.SaveAs(name+"/efficiency_d2_vs_pt.png")
c_eff.Draw()

In [61]:
c_eff = TCanvas("c_eff", "B final state daughter Efficiency", 700, 500)
num = f["d3_num_pt"]
den = f["d3_den_pt"]
eff = ROOT.TEfficiency(num, den)
eff.SetTitle("("+decay+")"+"Efficiency final state B-daughter vs pt; p_{t} [GeV]; Efficiency")
eff.Draw()
c_eff.SaveAs(name+"/efficiency_d3_vs_pt.png")
c_eff.Draw()

In [62]:
c_eff = TCanvas("c_eff", "D Efficiency pt", 700, 500)
num = f["D_num_pt"]
den = f["D_den_pt"]
eff = ROOT.TEfficiency(num, den)
eff.SetTitle("("+decay+")"+"Efficiency D vs pt; p_{t} [Gev]; Efficiency")
eff.Draw()
c_eff.SaveAs(name+"/efficiency_D_vs_pt.png")
c_eff.Draw()

In [63]:
c_eff = TCanvas("c_eff", "B Efficiency pt", 700, 500)
num = f["B_num_pt"]
den = f["B_den_pt"]
eff = ROOT.TEfficiency(num, den)
eff.SetTitle("("+decay+")"+"Efficiency B vs pt; p_{t} [Gev]; Efficiency")
eff.Draw()
c_eff.SaveAs(name+"/efficiency_B_vs_pt.png")
c_eff.Draw()

In [64]:
c0=TCanvas("c0","pt reco distributions",200,10,700,500)
c0.cd()
hd1=f["d1_pt"]
hd2=f["d2_pt"]
hd3=f["d3_pt"]
print(hd1.Integral())
print(hd2.Integral())
print(hd3.Integral())
hd1.SetLineColor(2)
hd2.SetLineColor(4)
hd3.SetLineColor(6)
hd1.SetStats(0) 
hd1.Draw()
hd2.Draw("same")
hd3.Draw("same")
hd1.SetTitle("("+decay+")"+"Pt reco distributions")
hd1.GetXaxis().SetTitle("p_{t} [Gev/c]")
hd1.GetYaxis().SetTitle("entries")
legend=TLegend(0.5,0.8,0.85,0.6)
legend.AddEntry(hd1,"lower p_{t} D-daughter p_{t}")
legend.AddEntry(hd2,"higher p_{t} D-daughter p_{t}")
legend.AddEntry(hd3,"final state B-daughter p_{t}")
ROOT.gStyle.SetLegendTextSize(0.05)
legend.Draw()
c0.Draw()
c0.SaveAs(name+"/pt_reco_distributions.png")

688181.0
746214.0
735524.0


In [65]:
c0=TCanvas("c0","eta reco distributions",200,10,700,500)
c0.cd()
hd1=f["d1_eta"]
hd2=f["d2_eta"]
hd3=f["d3_eta"]
print(hd1.Integral())
print(hd2.Integral())
print(hd3.Integral())
hd1.SetLineColor(2)
hd2.SetLineColor(4)
hd3.SetLineColor(6)
hd1.SetStats(0) 
hd1.Draw()
hd2.Draw("same")
hd3.Draw("same")
hd1.SetTitle("("+decay+")"+"Eta reco distributions")
hd1.GetXaxis().SetTitle("Eta")
hd1.GetYaxis().SetTitle("entries")
legend=TLegend(0.325, 0.2, 0.675, 0.4)
legend.AddEntry(hd1,"lower pt D-daughter eta")
legend.AddEntry(hd2,"higher pt D-daughter eta")
legend.AddEntry(hd3,"final state B-daughter eta")
ROOT.gStyle.SetLegendTextSize(0.05)
legend.Draw()
c0.Draw()
c0.SaveAs(name+"/eta_reco_distributions.png")

688234.0
747218.0
737301.0


In [66]:
c0=TCanvas("c0","phi reco distributions",200,10,700,500)
c0.cd()
hd1=f["d1_phi"]
hd2=f["d2_phi"]
hd3=f["d3_phi"]
print(hd1.Integral())
print(hd2.Integral())
print(hd3.Integral())
hd1.SetLineColor(2)
hd2.SetLineColor(4)
hd3.SetLineColor(6)
hd1.SetStats(0) 
hd1.Draw()
hd2.Draw("same")
hd3.Draw("same")
hd1.SetTitle("("+decay+")"+"Phi reco distributions")
hd1.GetXaxis().SetTitle("phi [rad]")
hd1.GetYaxis().SetTitle("entries")
legend=TLegend(0.325, 0.2, 0.675, 0.4)
legend.AddEntry(hd1,"lower pt D-daughter phi")
legend.AddEntry(hd2,"higher pt D-daughter phi")
legend.AddEntry(hd3,"final state B-daughter phi")
ROOT.gStyle.SetLegendTextSize(0.05)
legend.Draw()
c0.Draw()
c0.SaveAs(name+"/phi_reco_distributions.png")

688234.0
747218.0
737301.0


In [67]:
c = ROOT.TCanvas("c", "D invariant mass", 700, 500)
hist = f["D_M"]
hist.SetLineColor(ROOT.kBlue)
hist.SetTitle("("+decay+")"+"D invariant mass; Invariant mass [GeV/c^2]; Entries")
hist.Draw()
c.SaveAs(name+"/D_M.png")
c.Draw()

In [68]:
c = ROOT.TCanvas("c", "B invariant mass", 700, 500)
hist = f["B_M"]
hist.SetLineColor(ROOT.kBlue)
hist.SetTitle("("+decay+")"+"B invariant mass; Invariant mass [GeV/c^2]; Entries")
hist.Draw()
c.SaveAs(name+"/B_M.png")
c.Draw()

In [69]:
from ROOT import TLine
c = ROOT.TCanvas("c", "dr distribution d1", 700, 500)
hist = f["d1_dr"]
hist.SetLineColor(ROOT.kBlue)
hist.SetTitle("("+decay+")"+"lower pt D-daughter dr distribution;Angular Distance; Entries")
hist.Draw()
x_line = 0.01
y_min = hist.GetMinimum()
y_max = hist.GetMaximum()
line = TLine(x_line, y_min, x_line, y_max)
line.SetLineColor(ROOT.kRed)
line.SetLineWidth(2)
line.SetLineStyle(2)
line.Draw("same")
c.SaveAs(name+"/d1_dr.png")
c.Draw()

In [70]:
c = ROOT.TCanvas("c", "dr distribution d2", 700, 500)
hist = f["d2_dr"]
hist.SetLineColor(ROOT.kBlue)
hist.SetTitle("("+decay+")"+"higher pt D-daughter dr distribution;Angular Distance; Entries")
hist.Draw()
x_line = 0.01
y_min = hist.GetMinimum()
y_max = hist.GetMaximum()
line = TLine(x_line, y_min, x_line, y_max)
line.SetLineColor(ROOT.kRed)
line.SetLineWidth(2)
line.SetLineStyle(2)
line.Draw("same")
c.SaveAs(name+"/d2_dr.png")
c.Draw()

In [71]:
c = ROOT.TCanvas("c", "dr distribution d3", 700, 500)
hist = f["d3_dr"]
hist.SetLineColor(ROOT.kBlue)
hist.SetTitle("("+decay+")"+"final state B-daughter dr distribution;Angular Distance; Entries")
hist.Draw()
x_line = 0.01
y_min = hist.GetMinimum()
y_max = hist.GetMaximum()
line = TLine(x_line, y_min, x_line, y_max)
line.SetLineColor(ROOT.kRed)
line.SetLineWidth(2)
line.SetLineStyle(2)
line.Draw("same")
c.SaveAs(name+"/d3_dr.png")
c.Draw()

In [72]:
c = ROOT.TCanvas("c", "dr distribution D", 700, 500)
hist = f["D_dr"]
hist.SetLineColor(ROOT.kBlue)
hist.SetTitle("("+decay+")"+"D_dr;Angular Distance; Entries")
hist.Draw()
c.SaveAs(name+"/D_dr.png")
c.Draw()

In [73]:
c = ROOT.TCanvas("c", "dr distribution B", 700, 500)
hist = f["B_dr"]
hist.SetLineColor(ROOT.kBlue)
hist.SetTitle("("+decay+")"+"B_dr;Angular Distance; Entries")
hist.Draw()
c.SaveAs(name+"/B_dr.png")
c.Draw()

In [74]:
c0=TCanvas("c0","pt gen distributions",200,10,700,500)
c0.cd()
hd1=f["d1_den_pt"]
hd2=f["d2_den_pt"]
hd3=f["d3_den_pt"]
print(hd1.Integral())
print(hd2.Integral())
print(hd3.Integral())
hd1.SetLineColor(2)
hd2.SetLineColor(4)
hd3.SetLineColor(6)
hd1.SetStats(0) 
hd1.Draw()
hd2.Draw("same")
hd3.Draw("same")
hd1.SetTitle("("+decay+")"+"Pt gen distributions")
hd1.GetXaxis().SetTitle("p_{t} [Gev/c]")
hd1.GetYaxis().SetTitle("entries")
legend=TLegend(0.5,0.8,0.85,0.6)
legend.AddEntry(hd1,"lower p_{t} D-daughter p_{t}")
legend.AddEntry(hd2,"higher p_{t} D-daughter p_{t}")
legend.AddEntry(hd3,"final state B-daughter p_{t}")
ROOT.gStyle.SetLegendTextSize(0.05)
legend.Draw()
c0.Draw()
c0.SaveAs(name+"/pt_gen_distributions.png")

816883.0
815826.0
814975.0


In [75]:
c0=TCanvas("c0","phi gen distributions",200,10,700,500)
c0.cd()
hd1=f["d1_gen_phi"]
hd2=f["d2_gen_phi"]
hd3=f["d3_gen_phi"]
print(hd1.Integral())
print(hd2.Integral())
print(hd3.Integral())
hd1.SetLineColor(2)
hd2.SetLineColor(4)
hd3.SetLineColor(6)
hd1.SetStats(0) 
hd1.Draw()
hd2.Draw("same")
hd3.Draw("same")
hd1.SetTitle("("+decay+")"+"Phi gen distributions")
hd1.GetXaxis().SetTitle("phi [rad]")
hd1.GetYaxis().SetTitle("entries")
legend=TLegend(0.325, 0.2, 0.675, 0.4)
legend.AddEntry(hd1,"lower pt D-daughter phi")
legend.AddEntry(hd2,"higher pt D-daughter phi")
legend.AddEntry(hd3,"final state B-daughter phi")
ROOT.gStyle.SetLegendTextSize(0.05)
legend.Draw()
c0.Draw()
c0.SaveAs(name+"/phi_gen_distributions.png")

816941.0
816941.0
816941.0


In [76]:
c0=TCanvas("c0","eta gen distributions",200,10,700,500)
c0.cd()
hd1=f["d1_gen_eta"]
hd2=f["d2_gen_eta"]
hd3=f["d3_gen_eta"]
print(hd1.Integral())
print(hd2.Integral())
print(hd3.Integral())
hd1.SetLineColor(2)
hd2.SetLineColor(4)
hd3.SetLineColor(6)
hd1.SetStats(0) 
hd1.Draw()
hd2.Draw("same")
hd3.Draw("same")
hd1.SetTitle("("+decay+")"+"Eta gen distributions")
hd1.GetXaxis().SetTitle("Eta")
hd1.GetYaxis().SetTitle("entries")
legend=TLegend(0.325, 0.2, 0.675, 0.4)
legend.AddEntry(hd1,"lower pt D-daughter eta")
legend.AddEntry(hd2,"higher pt D-daughter eta")
legend.AddEntry(hd3,"final state B-daughter eta")
ROOT.gStyle.SetLegendTextSize(0.05)
legend.Draw()
c0.Draw()
c0.SaveAs(name+"/eta_gen_distributions.png")

816941.0
816941.0
816941.0


In [77]:
c = ROOT.TCanvas("c", "eta_phi_distribution_reco_final_state_1", 700, 500)
hist = f["eta_phi_reco_final_state_1"]
hist.SetTitle("("+decay+")"+"eta_phi_distribution_reco_lower_pt_D-daughter; Eta; Phi")
hist.Draw()
c.SaveAs(name+"/eta_phi_distribution_reco_final_state_1.png")
c.Draw()

In [78]:
c = ROOT.TCanvas("c", "eta_phi_distribution_reco_final_state_2", 700, 500)
hist = f["eta_phi_reco_final_state_2"]
hist.SetTitle("("+decay+")"+"eta_phi_distribution_reco_higher_pt_D-daughter; Eta; Phi")
hist.Draw()
c.SaveAs(name+"/eta_phi_distribution_reco_final_state_2.png")
c.Draw()

In [79]:
c = ROOT.TCanvas("c", "eta_phi_distribution_reco_final_state_3", 700, 500)
hist = f["eta_phi_reco_final_state_3"]
hist.SetTitle("("+decay+")"+"eta_phi_distribution_reco_final_state_B-daughter; Eta; Phi")
hist.Draw()
c.SaveAs(name+"/eta_phi_distribution_reco_final_state_3.png")
c.Draw()

In [80]:
c = ROOT.TCanvas("c", "eta_phi_distribution_gen_final_state_1", 700, 500)
hist = f["eta_phi_gen_final_state_1"]
hist.SetTitle("("+decay+")"+"eta_phi_distribution_reco_lower_pt_D-daughter; Eta; Phi")
hist.Draw()
c.SaveAs(name+"/eta_phi_distribution_reco_final_state_1.png")
c.Draw()

In [81]:
c = ROOT.TCanvas("c", "eta_phi_distribution_gen_final_state_2", 700, 500)
hist = f["eta_phi_gen_final_state_2"]
hist.SetTitle("("+decay+")"+"eta_phi_distribution_gen_higher_pt_D-daughter;Eta; Phi")
hist.Draw()
c.SaveAs(name+"/eta_phi_distribution_gen_final_state_2.png")
c.Draw()

In [82]:
c = ROOT.TCanvas("c", "eta_phi_distribution_gen_final_state_3", 700, 500)
hist = f["eta_phi_gen_final_state_3"]
hist.SetTitle("("+decay+")"+"eta_phi_distribution_gen_final_state_B-daughter; Eta; Phi")
hist.Draw()
c.SaveAs(name+"/eta_phi_distribution_gen_final_state_3.png")
c.Draw()

In [83]:
c0=TCanvas("c0","B pt distribution, reco vs gen",200,10,700,500)
c0.cd()
hd1=f["B_den_pt"]
hd2=f["B_num_pt"]
hd1.SetLineColor(2)
hd2.SetLineColor(4)
hd1.SetStats(0) 
hd1.Draw()
hd2.Draw("same")
hd1.SetTitle("("+decay+")"+"B pt distribution, reco vs gen")
hd1.GetXaxis().SetTitle("pt [Gev]")
hd1.GetYaxis().SetTitle("entries")
legend=TLegend(0.7,0.8,0.85,0.6)
legend.AddEntry(hd1,"gen B")
legend.AddEntry(hd2,"matched gen B")
ROOT.gStyle.SetLegendTextSize(0.05)
legend.Draw()
c0.Draw()
c0.SaveAs(name+"/B_pt_reco_vs_gen.png")

In [84]:
c0=TCanvas("c0","D pt distribution, reco vs gen",200,10,700,500)
c0.cd()
hd1=f["D_den_pt"]
hd2=f["D_num_pt"]
hd1.SetLineColor(2)
hd2.SetLineColor(4)
hd1.SetStats(0) 
hd1.Draw()
hd2.Draw("same")
hd1.SetTitle("("+decay+")"+"D pt distribution, reco vs gen")
hd1.GetXaxis().SetTitle("pt [Gev]")
hd1.GetYaxis().SetTitle("entries")
legend=TLegend(0.7,0.8,0.85,0.6)
legend.AddEntry(hd1,"gen D")
legend.AddEntry(hd2,"matched gen D")
ROOT.gStyle.SetLegendTextSize(0.05)
legend.Draw()
c0.Draw()
c0.SaveAs(name+"/D_pt_reco_vs_gen.png")

In [ ]:
"""
c_eff = TCanvas("c_eff", "d1 Efficiency d", 700, 500)
num = f["d1_num_d"]
den = f["d1_den_d"]
eff = ROOT.TEfficiency(num, den)
eff.SetTitle("Efficiency d1 vs d; d [cm]; Efficiency")
eff.Draw()
c_eff.SaveAs(name+"/efficiency_d1_vs_d.png")
c_eff.Draw()
"""

In [ ]:
"""
c_eff = TCanvas("c_eff", "d2 Efficiency d", 700, 500)
num = f["d2_num_d"]
den = f["d2_den_d"]
eff = ROOT.TEfficiency(num, den)
eff.SetTitle("Efficiency d2 vs d; d [cm]; Efficiency")
eff.Draw()
c_eff.SaveAs(name+"/efficiency_d2_vs_d.png")
c_eff.Draw()
"""

In [ ]:
"""
c_eff = TCanvas("c_eff", "d3 Efficiency d", 700, 500)
num = f["d3_num_d"]
den = f["d3_den_d"]
eff = ROOT.TEfficiency(num, den)
eff.SetTitle("Efficiency d3 vs d; d [cm]; Efficiency")
eff.Draw()
c_eff.SaveAs(name+"/efficiency_d3_vs_d.png")
c_eff.Draw()
"""